In [20]:
from torch import nn
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [4]:
def one_hot_encoding(y_array):
    from sklearn.preprocessing import LabelBinarizer
    enc = LabelBinarizer()
    return enc.fit_transform(y_array)

def load_data_from_csv(file_name):
    
    df = pd.read_csv(file_name).astype(float)
    print(f"Data for prediction with size = {df.shape[0]}")
    
    test_np_data = df.values.astype(np.float) / 255.
    
    test_tensor = torch.from_numpy(test_np_data)
    tensor_ds_test = torch.utils.data.TensorDataset(test_tensor)
    tensor_dl_test = torch.utils.data.DataLoader(tensor_ds_test)
    
    return tensor_dl_test

In [5]:
test_dl = load_data_from_csv(file_name='mnist_dataset\\test.csv')

Data for prediction with size = 28000


#### Use same model

In [6]:
class ClassificationNet(nn.Module):
    
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        
        self.hidden = nn.Linear(input_size, hidden_size)
        self.output = nn.Linear(hidden_size, output_size)
        self.activation = nn.Softmax()
        
    def forward(self, x):
        h_out = self.hidden(x)
        y_out = self.output(h_out)
        
        return self.activation(y_out)

In [10]:
model = torch.load('mnist_classifier.model')
model = model.to(torch.device(0))

In [21]:
image_ids = []
predictions = []

for image_id, img_x in enumerate(test_dl):
    image_ids.append(image_id+1)
    predictions.append(torch.argmax(model(img_x[0].cuda())).item())

print("Size of predictions = ", len(predictions))

<ipython-input-6-c0b960d54e2b>:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.activation(y_out)


Size of predictions =  28000


In [24]:
results = pd.DataFrame({'ImageId': image_ids, 'Label': predictions})
results.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [25]:
results.to_csv('mnist_results.csv', index=False)